In [1]:
import numpy as np
import xgboost as xgb
import os
from tensorflow.keras.datasets import mnist
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import threading
import time
import subprocess
import psutil
import tempfile
import re

2024-07-31 20:31:44.480583: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import xgboost as xgb
import onnxmltools
import onnxruntime as ort
from skl2onnx.common.data_types import FloatTensorType
import numpy as np
from scipy.sparse import issparse
from sklearn.metrics import accuracy_score

loaded_model = xgb.XGBClassifier()
loaded_model.load_model('./models_train/mnist_xgboost_model.json')

# 指定模型输入的数据类型
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]
# 转换模型
onnx_model = onnxmltools.convert_xgboost(loaded_model, initial_types=initial_type)
onnx_model_path = 'imdb_xgboost_model.onnx'
onnxmltools.utils.save_model(onnx_model, onnx_model_path)


NameError: name 'X_train' is not defined

In [3]:
# 加载 MNIST 数据集
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 将图像数据从 3D 重塑为 2D (每个图像为一行，像素为列)
X_train_2d = X_train.reshape(-1, 28*28)
X_test_2d = X_test.reshape(-1, 28*28)

# 将数据类型转换为 float32
X_train_2d = X_train_2d.astype('float32') / 255
X_test_2d = X_test_2d.astype('float32') / 255

# XGBoost 对象
model = xgb.XGBClassifier(
    objective='multi:softmax',  # 目标函数
    num_class=10,               # 类别数，与 MNIST 的标签数量相同
    n_estimators=100,           # 树的个数
    max_depth=6,                # 树的深度
    learning_rate=0.1           # 学习速率
)

# 训练模型
model.fit(X_train_2d, y_train)

# 预测测试集
y_pred = model.predict(X_test_2d)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy: {:.2f}%".format(accuracy * 100))

# loaded_model = xgb.XGBClassifier()
# loaded_model.load_model('./models_train/mnist_xgboost_model.json')
model.save_model('./models_train/mnist_xgboost_model.json')

Test accuracy: 97.02%


## XGB Mnist JSON

In [15]:
# 初始化全局变量
thread_output = {}
duration = 0
inference_duration = 0

# 定义保存模型的函数
def save_model(stop_event, model, X_test_tfidf):
    global duration
    global inference_duration

    start_time = time.time()
    for i in range(10):
        model.save_model('mnist_xgboost_model.json')
    end_time = time.time()

    duration = end_time - start_time
    print(f'Time taken to save model: {duration:.4f} seconds')

    start_time_inference = time.time()
    # 进行推理
    model.predict(X_test_tfidf)
    end_time_inference = time.time()
    for i in range(64):
        inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()
                    process.wait()
        
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 创建和启动线程
stop_event = threading.Event()

# 加载XGBoost模型
loaded_model = xgb.XGBClassifier()
loaded_model.load_model('./models_train/mnist_xgboost_model.json')

# save_model(stop_event, loaded_model, X_test_tfidf)

thread1 = threading.Thread(target=save_model, args=(stop_event, loaded_model, X_test_2d))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 和 "GPU Power" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

output_file_name = './mnist_models/output_xgboost/output-mnist-xgboost-json.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")

Subprocess started.
Time taken to save model: 1.6805 seconds
Time taken for inference on samples: 0.0720 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_xgboost/output-mnist-xgboost-json.txt
148
['CPU Power: 14649 mW', 'GPU Power: 0 mW', 'CPU Power: 8502 mW', 'GPU Power: 0 mW', 'CPU Power: 4013 mW', 'GPU Power: 0 mW', 'CPU Power: 15533 mW', 'GPU Power: 64 mW', 'CPU Power: 14568 mW', 'GPU Power: 0 mW', 'CPU Power: 15664 mW', 'GPU Power: 0 mW', 'CPU Power: 14438 mW', 'GPU Power: 58 mW', 'CPU Power: 16319 mW', 'GPU Power: 53 mW', 'CPU Power: 13937 mW', 'GPU Power: 0 mW', 'CPU Power: 6723 mW', 'GPU Power: 0 mW', 'CPU Power: 12890 mW', 'GPU Power: 0 mW', 'CPU Power: 15025 mW', 'GPU Power: 0 mW', 'CPU Power: 15713 mW', 'GPU Power: 0 mW', 'CPU Power: 14890 mW', 'GPU Power: 57 mW', 'CPU Power: 14143 mW', 'GPU Power: 0 mW', 'CPU Power: 16472 mW', 'GPU Power: 0 mW', 'CPU Power: 9972 mW', 'GPU Power: 0 mW', 'CPU Power: 4199 mW', 'GPU Power: 0 mW',

## XGB Mnist ONNX

In [16]:
import threading
import time
import subprocess
import tempfile
import os
import re
import psutil
import numpy as np
import xgboost as xgb
import onnxmltools
from onnxmltools.convert.xgboost import convert
from skl2onnx.common.data_types import FloatTensorType
from onnxruntime import InferenceSession

# 初始化全局变量
thread_output = {}
duration = 0
inference_duration = 0

# 定义保存模型的函数
def save_model(stop_event, model, X_test_2d):
    global duration
    global inference_duration

    initial_type = [('float_input', FloatTensorType([None, X_test_2d.shape[1]]))]
    # 转换模型
    onnx_model = convert(loaded_model, initial_types=initial_type)
    onnx_model_path = 'imdb_xgboost_model.onnx'

    start_time = time.time()
    for i in range(10):
        onnxmltools.utils.save_model(onnx_model, onnx_model_path)
    end_time = time.time()

    duration = end_time - start_time
    print(f'Time taken to save model: {duration:.4f} seconds')

    start_time_inference = time.time()
    # 进行推理
    session = InferenceSession(onnx_model_path)
    input_name = session.get_inputs()[0].name
    for i in range(64):
        predictions = session.run(None, {input_name: X_test_2d.astype(np.float32)})
    end_time_inference = time.time()

    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()
                    process.wait()
        
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 创建和启动线程
stop_event = threading.Event()

# 加载XGBoost模型
loaded_model = xgb.XGBClassifier()
loaded_model.load_model('./models_train/mnist_xgboost_model.json')

# save_model(stop_event, loaded_model, X_test_tfidf)

thread1 = threading.Thread(target=save_model, args=(stop_event, loaded_model, X_test_2d))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 和 "GPU Power" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

output_file_name = './mnist_models/output_xgboost/output-mnist-xgboost-onnx.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")


Subprocess started.
Time taken to save model: 0.2933 seconds
Time taken for inference on samples: 9.9839 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_xgboost/output-mnist-xgboost-onnx.txt
1318
['CPU Power: 5456 mW', 'GPU Power: 345 mW', 'CPU Power: 4385 mW', 'GPU Power: 131 mW', 'CPU Power: 3521 mW', 'GPU Power: 128 mW', 'CPU Power: 5798 mW', 'GPU Power: 58 mW', 'CPU Power: 5331 mW', 'GPU Power: 0 mW', 'CPU Power: 5436 mW', 'GPU Power: 52 mW', 'CPU Power: 5545 mW', 'GPU Power: 52 mW', 'CPU Power: 5081 mW', 'GPU Power: 105 mW', 'CPU Power: 5157 mW', 'GPU Power: 0 mW', 'CPU Power: 4518 mW', 'GPU Power: 0 mW', 'CPU Power: 2827 mW', 'GPU Power: 0 mW', 'CPU Power: 5261 mW', 'GPU Power: 0 mW', 'CPU Power: 5283 mW', 'GPU Power: 0 mW', 'CPU Power: 5029 mW', 'GPU Power: 0 mW', 'CPU Power: 6166 mW', 'GPU Power: 51 mW', 'CPU Power: 6834 mW', 'GPU Power: 98 mW', 'CPU Power: 7216 mW', 'GPU Power: 0 mW', 'CPU Power: 8331 mW', 'GPU Power: 0 mW', 'C

## XGBOOST MNIST PY

In [9]:
import threading
import time
import subprocess
import tempfile
import os
import re
import psutil
import numpy as np
import xgboost as xgb
import hummingbird.ml
import torch
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist  # 确保导入 MNIST 数据集

In [17]:
# 初始化全局变量
thread_output = {}
duration = 0
inference_duration = 0

# 定义保存模型的函数
def save_model(stop_event, model, X_test_2d):
    global duration
    global inference_duration

    # 将XGBoost模型转换为PyTorch模型
    start_time = time.time()
    for i in range(5):
        pytorch_model = hummingbird.ml.convert(model, 'pytorch')
        # 保存PyTorch模型
        torch.save(pytorch_model.model, './mnist_xgboost_model.pth')
    end_time = time.time()

    duration = end_time - start_time
    print(f'Time taken to save model: {duration:.4f} seconds')

    # 准备输入数据
    start_time_inference = time.time()
    # 将测试集数据转换为PyTorch张量
    X_test_torch = torch.tensor(X_test_2d, dtype=torch.float32)

    # 使用转换后的PyTorch模型进行预测
    # 进行推理
    for i in range(32):  # 1*10
        y_pred_torch = pytorch_model.predict(X_test_torch)

    end_time_inference = time.time()

    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {X_test_torch.shape[0]} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()
                    process.wait()
        
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 创建和启动线程
stop_event = threading.Event()

# 加载XGBoost模型
loaded_model = xgb.XGBClassifier()
loaded_model.load_model('./models_train/mnist_xgboost_model.json')

# 启动并监控线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model, X_test_2d))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 和 "GPU Power" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

output_file_name = './mnist_models/output_xgboost/output-mnist-xgboost-pth.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")

Subprocess started.
Time taken to save model: 19.5147 seconds
Time taken for inference on 10000 samples: 57.4827 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_xgboost/output-mnist-xgboost-pth.txt
8218
['CPU Power: 7336 mW', 'GPU Power: 0 mW', 'CPU Power: 7249 mW', 'GPU Power: 124 mW', 'CPU Power: 5388 mW', 'GPU Power: 320 mW', 'CPU Power: 4195 mW', 'GPU Power: 0 mW', 'CPU Power: 5528 mW', 'GPU Power: 0 mW', 'CPU Power: 5811 mW', 'GPU Power: 0 mW', 'CPU Power: 5723 mW', 'GPU Power: 0 mW', 'CPU Power: 6879 mW', 'GPU Power: 94 mW', 'CPU Power: 6241 mW', 'GPU Power: 197 mW', 'CPU Power: 3932 mW', 'GPU Power: 52 mW', 'CPU Power: 4959 mW', 'GPU Power: 0 mW', 'CPU Power: 5484 mW', 'GPU Power: 0 mW', 'CPU Power: 6255 mW', 'GPU Power: 0 mW', 'CPU Power: 6378 mW', 'GPU Power: 54 mW', 'CPU Power: 6401 mW', 'GPU Power: 164 mW', 'CPU Power: 6629 mW', 'GPU Power: 53 mW', 'CPU Power: 6230 mW', 'GPU Power: 0 mW', 'CPU Power: 5102 mW', 'GPU Power: 57 

## Model RF-Mnist

In [6]:
import joblib
import onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as rt
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.datasets import mnist
from scipy.sparse import hstack

In [7]:
# 加载 MNIST 数据集
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 将图像数据从 3D 重塑为 2D (每个图像为一行，像素为列)
X_train_2d = X_train.reshape(-1, 28*28)
X_test_2d = X_test.reshape(-1, 28*28)

# 将数据类型转换为 float32 并进行归一化处理
X_train_2d = X_train_2d.astype('float32') / 255
X_test_2d = X_test_2d.astype('float32') / 255

# 创建随机森林分类器
# n_estimators 表示使用的决策树数量，更多的树会提高模型的性能和稳定性，但也会增加计算成本
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# 训练模型
clf.fit(X_train_2d, y_train)

# 预测测试集
y_pred = clf.predict(X_test_2d)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy * 100:.2f}%')

# 保存训练好的模型到文件
model_filename = ('./models_train/mnist_random_forest_model.joblib')
dump(clf, model_filename)
print(f"Model saved to {model_filename}")

NameError: name 'mnist' is not defined

## MNIST RF Joblib

In [12]:
import joblib
# 初始化全局变量
thread_output = {}
duration = 0
inference_duration = 0

# 定义保存模型的函数
def save_model(stop_event, model, X_test_tfidf):
    global duration
    global inference_duration

    start_time = time.time()
    for i in range(10):
        joblib.dump(model, 'mnist_random_forest_model.joblib')
    end_time = time.time()

    duration = end_time - start_time
    print(f'Time taken to save model: {duration:.4f} seconds')

    start_time_inference = time.time()
    # 进行推理
    for i in range(1): # 1*10
        model.predict(X_test_tfidf)
    end_time_inference = time.time()

    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()
                    process.wait()
        
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 创建和启动线程
stop_event = threading.Event()

# 加载RF模型
# 加载随机森林模型
loaded_model = joblib.load('./models_train/mnist_random_forest_model.joblib')

thread1 = threading.Thread(target=save_model, args=(stop_event, loaded_model, X_test_2d))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 和 "GPU Power" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

output_file_name = './mnist_models/output_rf/output-mnist-rf-joblib.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")

Subprocess started.
Time taken to save model: 6.4584 seconds
Time taken for inference on samples: 1.1318 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_rf/output-mnist-rf-joblib.txt
814
['CPU Power: 5026 mW', 'GPU Power: 0 mW', 'CPU Power: 13113 mW', 'GPU Power: 0 mW', 'CPU Power: 11643 mW', 'GPU Power: 0 mW', 'CPU Power: 11826 mW', 'GPU Power: 0 mW', 'CPU Power: 11036 mW', 'GPU Power: 60 mW', 'CPU Power: 3882 mW', 'GPU Power: 0 mW', 'CPU Power: 3369 mW', 'GPU Power: 0 mW', 'CPU Power: 4128 mW', 'GPU Power: 0 mW', 'CPU Power: 4821 mW', 'GPU Power: 0 mW', 'CPU Power: 6332 mW', 'GPU Power: 123 mW', 'CPU Power: 7881 mW', 'GPU Power: 0 mW', 'CPU Power: 5282 mW', 'GPU Power: 60 mW', 'CPU Power: 4194 mW', 'GPU Power: 0 mW', 'CPU Power: 2457 mW', 'GPU Power: 0 mW', 'CPU Power: 3460 mW', 'GPU Power: 0 mW', 'CPU Power: 2919 mW', 'GPU Power: 0 mW', 'CPU Power: 3689 mW', 'GPU Power: 0 mW', 'CPU Power: 2383 mW', 'GPU Power: 0 mW', 'CPU Power: 3433

### RF-MNIST onnx

In [13]:
import time
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as rt
import joblib

# 初始化全局变量
thread_output = {}
duration = 0
inference_duration = 0

# 定义保存模型的函数
def save_model(stop_event, model, X_test_tfidf):
    global duration
    global inference_duration

    # 指定模型输入的数据类型
    initial_type = [('float_input', FloatTensorType([None, X_train_2d.shape[1]]))]

    # 转换模型
    onnx_model = convert_sklearn(model, initial_types=initial_type)
    onnx_model_path = 'mnist_random_forest_model.onnx'

    # 保存 ONNX 模型
    start_time = time.time()
    for i in range(10):
        with open(onnx_model_path, "wb") as f:
            f.write(onnx_model.SerializeToString())
    end_time = time.time()

    duration = end_time - start_time
    print(f'Time taken to save model: {duration:.4f} seconds')

    # 使用 ONNX Runtime 进行推理
    start_time_inference = time.time()
    session = rt.InferenceSession(onnx_model_path)
    input_name = session.get_inputs()[0].name
    for i in range(1):  # Assuming 1*10 for inference as per original loop
        predictions = session.run(None, {input_name: X_test_tfidf.astype(np.float32)})[0]
    end_time_inference = time.time()

    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程


# 监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()
                    process.wait()
        
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 创建和启动线程
stop_event = threading.Event()

# 加载RF模型
# 加载随机森林模型
loaded_model = joblib.load('./models_train/mnist_random_forest_model.joblib')

thread1 = threading.Thread(target=save_model, args=(stop_event, loaded_model, X_test_2d))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 和 "GPU Power" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

output_file_name = './mnist_models/output_rf/output-mnist-rf-joblib.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")

Exception in thread Thread-26:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/env_name/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/env_name/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/tm/w_n0tvhd2jlds4ml_jrhf3yw0000gn/T/ipykernel_5710/4196126428.py", line 21, in save_model
NameError: name 'convert_sklearn' is not defined


Subprocess started.


KeyboardInterrupt: 

In [3]:
import threading
import time
import subprocess
import tempfile
import os
import re
import psutil
import numpy as np
import joblib
import onnxruntime as rt
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn

# 初始化全局变量
thread_output = {}
duration = 0
inference_duration = 0

# 定义保存模型的函数
def save_model(stop_event, model, X_test_tfidf):
    global duration
    global inference_duration

    # 将random_forest模型转换为ONNX模型
    start_time = time.time()
    for i in range(1):
        # 转换为ONNX格式
        initial_type = [('float_input', FloatTensorType([None, X_test_tfidf.shape[1]]))]
        onnx_model = convert_sklearn(model, initial_types=initial_type, target_opset=9)

        # 保存ONNX模型
        onnx_model_filename = 'mnist_random_forest_model.onnx'
        with open(onnx_model_filename, "wb") as f:
            f.write(onnx_model.SerializeToString())
        print(f"ONNX model saved to {onnx_model_filename}")
    end_time = time.time()

    duration = end_time - start_time
    print(f'Time taken to save model: {duration:.4f} seconds')

    start_time_inference = time.time()
    X_test_2d_array = X_test_2d.astype(np.float32)

    # 使用ONNX runtime进行推理
    sess = rt.InferenceSession(onnx_model.SerializeToString())
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name

    # 进行推理
    for i in range(10):  # 1*10
        predictions = sess.run([label_name], {input_name: X_test_2d_array})[0]
    # 进行推理
    end_time_inference = time.time()

    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {X_test_2d_array.shape[0]} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()
                    process.wait()
        
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 创建和启动线程
stop_event = threading.Event()

# 加载保存的模型
model_filename = './models_train/mnist_random_forest_model.joblib'
clf = joblib.load(model_filename)

thread1 = threading.Thread(target=save_model, args=(stop_event, clf, X_test_2d))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 和 "GPU Power" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

output_file_name = './mnist_models/output_random_forest/output-mnist-random_forest-onnx.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")


NameError: name 'X_test_2d' is not defined